In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr_w
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)# IMPORTING MODULES

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

wesad_label_type = "stai"
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 1.0


In [ ]:
importlib.reload(train)
importlib.reload(dr_w)

stai_scores, dim_scores_arousal, dim_scores_valence = train.Train_WESAD.get_labels(model_phases_wesad)
print(stai_scores)

In [39]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

# x = x[x['subject'] != 8.0]
# y = y[y['subject'] != 8.0]

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=True, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    
print("\n")

y_train:
1    1522
0     262
Name: label, dtype: int64
y_test:
0    55
1    15
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [12 58]
Model RF, Predictions: [0 1], [ 7 63]
Model XGB, Predictions: [0 1], [ 6 64]

[('mean_SCL', 0.36503474097110644), ('SCR_rate', 0.34790615148435833), ('lf_rr', 0.32803677158340006), ('hf_rr', -0.0401397663341208), ('bpm', -0.1510485020310252), ('lf_hf_ratio', -0.7198460377967504), ('sdnn', -1.1643226626094145), ('rmssd', -1.4541963127225255)]


[('mean_SCL', 0.14701225474592636), ('rmssd', 0.14203788980210136), ('sdnn', 0.13873860254798362), ('bpm', 0.13151954781110545), ('lf_hf_ratio', 0.12673388561434065), ('hf_rr', 0.11330793183973194), ('lf_rr', 0.1066974413926109), ('SCR_rate', 0.09395244624619976)]


[('sdnn', 0.13990077), ('lf_rr', 0.13073882), ('bpm', 0.12886035), ('mean_SCL', 0.12775761), ('rmssd', 0.12377887), ('hf_rr', 0.122103505), ('SCR_rate', 0.114196084), ('lf_hf_ratio', 0.11266404)]

LogReg accuracy over 1 rounds: 0.3
Model ev

In [38]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_wesad in enumerate(model_phases_wesad):
    print(f"WESAD PHASES {phases_wesad} " + "-"*50)
    phases_wesad = [phases_wesad]
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
    y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=True, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        
    print("\n")

WESAD PHASES Base --------------------------------------------------
y_train:
1    1522
0     262
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [ 3 11]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model RF, Predictions: [1], [14]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Model XGB, Predictions: [0 1], [ 3 11]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.

[('mean_SCL', 0.36503474097110644), ('SCR_rate', 0.34790615148435833), ('lf_rr', 0.32803677158340006), ('hf_rr', -0.0401397663341208), ('bpm', -0.1510485020310252), ('lf_hf_ratio', -0.7198460377967504), ('sdnn', -1.1643226626094145), ('rmssd', -1.4541963127225255)]


[('rmssd', 0.14465243674404887), ('mean_SCL', 0.1414314776254526), ('sdnn', 0.13943006077414585), ('bpm', 0.13609072209871043), ('lf_hf_ratio', 0

In [37]:
# TRAIN ON WESAD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)


print(f"WESAD PHASES {model_phases_wesad} " + "-"*50)
x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# drop subjects with noisy data
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.6:
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
print("\n")

WESAD PHASES ['Base', 'TSST', 'Medi_1', 'Fun', 'Medi_2'] --------------------------------------------------
Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
Ratio of negative to positive labels (0.1721419185282523) is under 0.333, oversampling negative class.
y_train:
0    55
1    18
Name: label, dtype: int64
y_test:
1    1522
0     506
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [1690  338]
Model RF, Predictions: [0 1], [ 980 1048]
Model XGB, Predictions: [0 1], [1594  434]

[('bpm', 2.466076057351408), ('mean_SCL', 1.2114356884190844), ('SCR_rate', 0.8648656984773081), ('lf_rr', 0.6582444865529865), ('rmssd', 0.2791441186833543), ('hf_rr', 0.06741097545500135), ('sdnn', -0.3490763037618447), ('lf_hf_ratio', -0.6621443262502791)]


[('bpm', 0.24656958881201205), ('hf_rr', 0.17485748157686978), ('mean_SCL', 0.15514085102849443), ('SCR_rate', 0.10465316604446082), ('rmssd', 0.09384439301663977), ('sdnn', 0.09103158218